In [13]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

# Define the path
base_path = Path.home() / "Portfolio_Exports"
print(f"Looking for files in: {base_path}")

# List available files
if base_path.exists():
    files = list(base_path.glob("*.csv")) + list(base_path.glob("*.xlsx")) + list(base_path.glob("*.parquet"))
    print("Files found in Portfolio_Exports:")
    for file in files:
        print(f"  - {file.name} ({file.stat().st_size / 1024**2:.2f} MB)")
else:
    print(f"Directory not found: {base_path}")
    # Try alternative paths
    alt_paths = [
        Path.home() / "Portfolio_Export",
        Path.home() / "Desktop" / "Portfolio_Exports",
        Path.home() / "Documents" / "Portfolio_Exports"
    ]
    for alt_path in alt_paths:
        if alt_path.exists():
            base_path = alt_path
            files = list(base_path.glob("*"))
            print(f"\nFound in alternative location: {base_path}")
            for file in files[:10]:  # Show first 10 files
                print(f"  - {file.name}")
            break

Looking for files in: C:\Users\user\Portfolio_Exports
Files found in Portfolio_Exports:
  - Cleaned_BPI_2019_Events.csv (412.20 MB)
  - Process_Bottleneck_Analysis.csv (0.00 MB)


In [14]:
def load_dataset(filename, sample_fraction=0.3, use_dask=False):
    """Load dataset with memory optimization"""
    filepath = base_path / filename
    
    if not filepath.exists():
        print(f"File not found: {filename}")
        # Try to find similar files
        similar_files = list(base_path.glob(f"*{filename.split('_')[0]}*"))
        if similar_files:
            filepath = similar_files[0]
            print(f"Using similar file: {filepath.name}")
        else:
            return None
    
    print(f"\n{'='*60}")
    print(f"LOADING: {filepath.name}")
    print(f"Size: {filepath.stat().st_size / 1024**2:.2f} MB")
    print('='*60)
    
    # Get file extension
    ext = filepath.suffix.lower()
    
    if use_dask and ext == '.csv':
        import dask.dataframe as dd
        print("Using Dask for large CSV file...")
        ddf = dd.read_csv(filepath)
        print(f"Columns: {ddf.columns.tolist()}")
        print(f"Number of partitions: {ddf.npartitions}")
        return ddf
    else:
        # First, sample to understand structure
        if ext == '.csv':
            # Get total rows for CSV
            with open(filepath, 'r') as f:
                total_rows = sum(1 for line in f) - 1  # Subtract header
            
            print(f"Total rows: {total_rows:,}")
            
            # Read sample to determine dtypes
            sample_size = min(10000, total_rows)
            df_sample = pd.read_csv(filepath, nrows=sample_size)
            
            # Optimize dtypes
            dtypes = {}
            for col in df_sample.columns:
                if df_sample[col].dtype == 'object':
                    unique_ratio = df_sample[col].nunique() / len(df_sample)
                    if unique_ratio < 0.5:
                        dtypes[col] = 'category'
                elif 'int' in str(df_sample[col].dtype):
                    col_min = df_sample[col].min()
                    col_max = df_sample[col].max()
                    if col_min > -128 and col_max < 128:
                        dtypes[col] = 'int8'
                    elif col_min > -32768 and col_max < 32768:
                        dtypes[col] = 'int16'
                    elif col_min > -2147483648 and col_max < 2147483647:
                        dtypes[col] = 'int32'
                    else:
                        dtypes[col] = 'int64'
            
            # Load data with sampling if large
            if total_rows > 500000 and sample_fraction < 1:
                print(f"Loading {sample_fraction*100:.0f}% sample ({int(total_rows*sample_fraction):,} rows)...")
                
                # Calculate rows to skip
                skip_rows = np.random.choice(
                    range(1, total_rows + 1),
                    size=int(total_rows * (1 - sample_fraction)),
                    replace=False
                )
                df = pd.read_csv(filepath, dtype=dtypes, skiprows=skip_rows)
            else:
                # Load in chunks
                chunks = []
                chunk_size = 100000
                print(f"Loading in chunks of {chunk_size:,}...")
                
                for i, chunk in enumerate(pd.read_csv(filepath, dtype=dtypes, chunksize=chunk_size)):
                    chunks.append(chunk)
                    if i % 10 == 0:
                        print(f"  Loaded {len(chunks)*chunk_size:,} rows...")
                
                df = pd.concat(chunks, ignore_index=True)
                
        elif ext == '.parquet':
            df = pd.read_parquet(filepath)
        elif ext == '.xlsx':
            df = pd.read_excel(filepath, nrows=100000)  # Limit for Excel
        else:
            print(f"Unsupported file format: {ext}")
            return None
        
        print(f"Loaded shape: {df.shape}")
        print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        return df

# Load both datasets
print("\n" + "="*80)
print("LOADING BOTH DATASETS")
print("="*80)

# Load Cleaned_BPI_2019_Events
bpi_df = load_dataset("Cleaned_BPI_2019_Events.csv", sample_fraction=0.2)

# Load Process_Bottleneck_Analysis
bottleneck_df = load_dataset("Process_Bottleneck_Analysis.csv", sample_fraction=1.0)  # Load full if smaller


LOADING BOTH DATASETS

LOADING: Cleaned_BPI_2019_Events.csv
Size: 412.20 MB
Total rows: 1,595,923
Loading 20% sample (319,184 rows)...
Loaded shape: (319185, 21)
Memory usage: 43.07 MB

LOADING: Process_Bottleneck_Analysis.csv
Size: 0.00 MB
Total rows: 42
Loading in chunks of 100,000...
  Loaded 100,000 rows...
Loaded shape: (42, 2)
Memory usage: 0.00 MB


In [16]:

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("COMPREHENSIVE BPI 2019 ANALYSIS - STEP BY STEP")
print("="*80)

COMPREHENSIVE BPI 2019 ANALYSIS - STEP BY STEP


In [18]:

# Step 1: Define file path and verify
print("\n" + "="*60)
print("STEP 1: LOCATE AND VERIFY DATASET")
print("="*60)

base_path = Path.home() / "Portfolio_Exports"
file_name = "Cleaned_BPI_2019_Events.csv"
file_path = base_path / file_name

if not file_path.exists():
    print(f"File not found at: {file_path}")
    # Search for the file
    files = list(base_path.glob("*BPI*2019*.csv"))
    if files:
        file_path = files[0]
        print(f"Found alternative: {file_path.name}")
    else:
        print("Searching in other locations...")
        search_paths = [
            Path.home() / "Portfolio_Export",
            Path.home() / "Desktop",
            Path.home() / "Documents"
        ]
        for path in search_paths:
            files = list(path.glob("**/*BPI*2019*.csv"))
            if files:
                file_path = files[0]
                print(f"Found at: {file_path}")
                break
else:
    print(f"File found: {file_path}")

print(f"File size: {file_path.stat().st_size / 1024**2:.2f} MB")


STEP 1: LOCATE AND VERIFY DATASET
File found: C:\Users\user\Portfolio_Exports\Cleaned_BPI_2019_Events.csv
File size: 412.20 MB


In [19]:

# Step 2: Efficient data loading with error handling
print("\n" + "="*60)
print("STEP 2: LOAD DATA WITH OPTIMIZED MEMORY USAGE")
print("="*60)

def load_bpi_data_safely(file_path, sample_size=100000):
    """Load BPI data with proper error handling"""
    try:
        # First, check the file structure
        print("Checking file structure...")
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            # Read first few lines
            header = f.readline().strip()
            second_line = f.readline().strip()
        
        columns = header.split(',')
        print(f"Found {len(columns)} columns")
        print(f"First few columns: {columns[:5]}...")
        
        # Create optimized dtypes dictionary
        print("\nDetermining optimal data types...")
        
        # Read a sample to understand data types
        sample_df = pd.read_csv(file_path, nrows=1000)
        
        dtypes_dict = {}
        for col in sample_df.columns:
            col_dtype = sample_df[col].dtype
            
            if pd.api.types.is_object_dtype(col_dtype):
                # Check if this looks like a date/time column
                col_lower = col.lower()
                if any(time_word in col_lower for time_word in ['time', 'date', 'timestamp', 'start', 'end', 'complete']):
                    # Don't convert datetime columns to categorical
                    dtypes_dict[col] = 'object'  # Keep as object, we'll convert later
                else:
                    # For other object columns, check cardinality
                    unique_ratio = sample_df[col].nunique() / len(sample_df)
                    if unique_ratio < 0.3:  # Less than 30% unique values
                        dtypes_dict[col] = 'category'
                    else:
                        dtypes_dict[col] = 'string'
                        
            elif pd.api.types.is_integer_dtype(col_dtype):
                # Downcast integers
                col_min = sample_df[col].min()
                col_max = sample_df[col].max()
                
                if col_min >= 0:  # Unsigned
                    if col_max < 256:
                        dtypes_dict[col] = 'uint8'
                    elif col_max < 65536:
                        dtypes_dict[col] = 'uint16'
                    elif col_max < 4294967296:
                        dtypes_dict[col] = 'uint32'
                    else:
                        dtypes_dict[col] = 'uint64'
                else:  # Signed
                    if col_min > -128 and col_max < 128:
                        dtypes_dict[col] = 'int8'
                    elif col_min > -32768 and col_max < 32768:
                        dtypes_dict[col] = 'int16'
                    elif col_min > -2147483648 and col_max < 2147483647:
                        dtypes_dict[col] = 'int32'
                    else:
                        dtypes_dict[col] = 'int64'
                        
            elif pd.api.types.is_float_dtype(col_dtype):
                dtypes_dict[col] = 'float32'
            else:
                dtypes_dict[col] = str(col_dtype)
        
        print(f"Determined optimal types for {len(dtypes_dict)} columns")
        
        # Load the data in chunks
        print("\nLoading data in chunks...")
        chunks = []
        chunk_size = 50000
        
        for i, chunk in enumerate(pd.read_csv(file_path, dtype=dtypes_dict, chunksize=chunk_size)):
            chunks.append(chunk)
            if (i + 1) % 10 == 0:
                print(f"   Loaded {(i + 1) * chunk_size:,} rows...")
        
        df = pd.concat(chunks, ignore_index=True)
        
        print(f"\nSuccessfully loaded {len(df):,} rows and {len(df.columns)} columns")
        print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        
        return df
        
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        # Try alternative loading method
        print("Trying alternative loading method...")
        try:
            df = pd.read_csv(file_path, nrows=sample_size)
            print(f"Loaded sample of {len(df):,} rows")
            return df
        except Exception as e2:
            print(f"Failed to load: {str(e2)}")
            return None

# Load the data
bpi_df = load_bpi_data_safely(file_path, sample_size=200000)

if bpi_df is None:
    print("Failed to load dataset. Exiting.")
    exit()


STEP 2: LOAD DATA WITH OPTIMIZED MEMORY USAGE
Checking file structure...
Found 21 columns
First few columns: ['activity', 'timestamp', 'ID', 'cCompany', 'cDocType']...

Determining optimal data types...
Determined optimal types for 21 columns

Loading data in chunks...
   Loaded 500,000 rows...
   Loaded 1,000,000 rows...
   Loaded 1,500,000 rows...

Successfully loaded 1,595,923 rows and 21 columns
Memory usage: 990.02 MB


In [21]:
# Step 3: Initial Data Inspection
print("\n" + "="*60)
print("STEP 3: INITIAL DATA INSPECTION")
print("="*60)

print(f"Dataset Shape: {bpi_df.shape[0]:,} rows × {bpi_df.shape[1]} columns")
print(f"Column Names:")
for i, col in enumerate(bpi_df.columns, 1):
    print(f"{i:2d}. {col}")

print("\nData Types:")
dtype_counts = bpi_df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"{dtype}: {count} columns")

print("\nFirst 5 rows:")
print(bpi_df.head())

print("\nColumn Details (First 10 columns):")
for col in bpi_df.columns[:10]:
    print(f"\nColumn: {col}")
    print(f" Type: {bpi_df[col].dtype}")
    print(f"Unique values: {bpi_df[col].nunique():,}")
    print(f"Missing values: {bpi_df[col].isnull().sum():,} ({bpi_df[col].isnull().sum()/len(bpi_df)*100:.1f}%)")
    
    if bpi_df[col].dtype == 'object' and bpi_df[col].nunique() < 20:
        print(f"Sample values: {bpi_df[col].unique()[:10]}")



STEP 3: INITIAL DATA INSPECTION
Dataset Shape: 1,595,923 rows × 21 columns
Column Names:
 1. activity
 2. timestamp
 3. ID
 4. cCompany
 5. cDocType
 6. cGR
 7. cGRbasedInvVerif
 8. cID
 9. cItem
10. cItemCat
11. cItemType
12. cPOID
13. cPurDocCat
14. cSpendAreaText
15. cSpendClassText
16. cSubSPendAreaText
17. cVendor
18. cVendorName
19. eCumNetWorth
20. idx
21. resource

Data Types:
object: 9 columns
float32: 4 columns
bool: 2 columns
uint64: 1 columns
category: 1 columns
string: 1 columns
category: 1 columns
category: 1 columns
category: 1 columns

First 5 rows:
                    activity            timestamp              ID  \
0     Vendor creates invoice  1948-01-26 22:59:00  65781719105536   
1  Vendor creates debit memo  1948-01-26 22:59:00  65777424138241   
2     Vendor creates invoice  1948-01-26 22:59:00  65777424138240   
3     Vendor creates invoice  1948-01-26 22:59:00  65794604007424   
4  Vendor creates debit memo  1948-01-26 22:59:00  65794604007425   

         cCo

In [22]:
# Step 4: Identify Key Columns
print("\n" + "="*60)
print("STEP 4: IDENTIFY KEY COLUMNS FOR PROCESS MINING")
print("="*60)

def identify_key_columns(df):
    """Identify key columns for process mining analysis"""
    key_columns = {
        'case_id': None,
        'activity': None,
        'timestamp': None,
        'resource': None,
        'cost': None
    }
    
    # Common patterns for each type of column
    patterns = {
        'case_id': ['case', 'id', 'nummer', 'nr', 'nummer', 'number', 'proc', 'process'],
        'activity': ['activity', 'event', 'action', 'task', 'step', 'handling'],
        'timestamp': ['time', 'date', 'timestamp', 'start', 'end', 'complete', 'created', 'modified'],
        'resource': ['resource', 'user', 'employee', 'person', 'role', 'team', 'group', 'org'],
        'cost': ['cost', 'amount', 'price', 'revenue', 'expense', 'value', 'money']
    }
    
    for col in df.columns:
        col_lower = col.lower()
        
        for key, pattern_list in patterns.items():
            for pattern in pattern_list:
                if pattern in col_lower:
                    if key_columns[key] is None:
                        key_columns[key] = col
                        print(f"Identified {key}: {col}")
                    break
    
    # If not found, try alternative methods
    if key_columns['case_id'] is None:
        # Look for columns with many unique values but not too many
        for col in df.columns:
            unique_ratio = df[col].nunique() / len(df)
            if 0.001 < unique_ratio < 0.5:  # Between 0.1% and 50% unique
                if df[col].dtype in ['object', 'string', 'category']:
                    key_columns['case_id'] = col
                    print(f"Guessing {key}: {col} (unique ratio: {unique_ratio:.3f})")
                    break
    
    return key_columns

key_columns = identify_key_columns(bpi_df)

print("\nSummary of Identified Key Columns:")
for key, value in key_columns.items():
    if value:
        print(f"{key:12}: {value} (Type: {bpi_df[value].dtype})")
    else:
        print(f"{key:12}: NOT FOUND")



STEP 4: IDENTIFY KEY COLUMNS FOR PROCESS MINING
Identified activity: activity
Identified timestamp: timestamp
Identified case_id: ID
Identified resource: resource

Summary of Identified Key Columns:
case_id     : ID (Type: uint64)
activity    : activity (Type: object)
timestamp   : timestamp (Type: object)
resource    : resource (Type: object)
cost        : NOT FOUND


In [23]:
# Step 5: Handle Categorical and DateTime Columns
print("\n" + "="*60)
print("STEP 5: DATA TYPE CONVERSION & CLEANING")
print("="*60)

def convert_data_types(df, key_columns):
    """Convert columns to proper data types with error handling"""
    print("Converting data types...")
    
    # 1. Convert timestamp column
    if key_columns['timestamp']:
        timestamp_col = key_columns['timestamp']
        print(f"   Converting '{timestamp_col}' to datetime...")
        
        try:
            # Handle categorical timestamps by converting to string first
            if pd.api.types.is_categorical_dtype(df[timestamp_col]):
                print(f"   Note: '{timestamp_col}' is categorical. Converting to string first...")
                df[timestamp_col] = df[timestamp_col].astype(str)
            
            # Convert to datetime
            df['_timestamp'] = pd.to_datetime(df[timestamp_col], errors='coerce')
            
            # Check success rate
            success_rate = (1 - df['_timestamp'].isnull().sum() / len(df)) * 100
            print(f"Successfully converted {success_rate:.1f}% of values")
            
            # Store original if needed
            df['_timestamp_original'] = df[timestamp_col]
            
        except Exception as e:
            print(f"   ❌ Error converting timestamp: {str(e)}")
    
    # 2. Handle categorical columns
    print("\n   Processing categorical columns...")
    for col in df.select_dtypes(include=['object', 'string']).columns:
        if col in ['_timestamp', '_timestamp_original']:
            continue
            
        unique_ratio = df[col].nunique() / len(df)
        if unique_ratio < 0.3:  # Good candidate for categorical
            try:
                # Ensure the column is ordered if we might use min/max
                if col in [key_columns['case_id'], key_columns['activity']]:
                    df[col] = df[col].astype('category').cat.as_ordered()
                    print(f"Converted '{col}' to ORDERED categorical ({df[col].nunique()} categories)")
                else:
                    df[col] = df[col].astype('category')
                    print(f"Converted '{col}' to categorical ({df[col].nunique()} categories)")
            except Exception as e:
                print(f"Error converting '{col}': {str(e)}")
    
    # 3. Handle numeric columns
    print("\n   Optimizing numeric columns...")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        try:
            original_memory = df[col].memory_usage(deep=True)
            
            # Downcast integers
            if pd.api.types.is_integer_dtype(df[col]):
                col_min = df[col].min()
                col_max = df[col].max()
                
                if col_min >= 0:  # Unsigned
                    if col_max < 256:
                        df[col] = df[col].astype('uint8')
                    elif col_max < 65536:
                        df[col] = df[col].astype('uint16')
                    elif col_max < 4294967296:
                        df[col] = df[col].astype('uint32')
                else:  # Signed
                    if col_min > -128 and col_max < 128:
                        df[col] = df[col].astype('int8')
                    elif col_min > -32768 and col_max < 32768:
                        df[col] = df[col].astype('int16')
                    elif col_min > -2147483648 and col_max < 2147483647:
                        df[col] = df[col].astype('int32')
                        
            # Downcast floats
            elif pd.api.types.is_float_dtype(df[col]):
                df[col] = df[col].astype('float32')
                
            new_memory = df[col].memory_usage(deep=True)
            if new_memory < original_memory:
                print(f"Optimized '{col}' memory usage")
                
        except Exception as e:
            print(f"Error optimizing '{col}': {str(e)}")
    
    return df

# Convert data types
bpi_df = convert_data_types(bpi_df, key_columns)

print(f"\nAfter optimization - Memory usage: {bpi_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")



STEP 5: DATA TYPE CONVERSION & CLEANING
Converting data types...
   Converting 'timestamp' to datetime...
Successfully converted 100.0% of values

   Processing categorical columns...
Converted 'activity' to ORDERED categorical (42 categories)
Converted 'timestamp' to categorical (167432 categories)
Converted 'cCompany' to categorical (4 categories)
Converted 'cID' to categorical (251734 categories)
Converted 'cSpendAreaText' to categorical (20 categories)
Converted 'cSpendClassText' to categorical (3 categories)
Converted 'cSubSPendAreaText' to categorical (135 categories)
Converted 'cVendor' to categorical (1975 categories)
Converted 'cVendorName' to categorical (1899 categories)
Converted 'resource' to categorical (628 categories)

   Optimizing numeric columns...

After optimization - Memory usage: 230.75 MB


In [24]:
# Step 6: Basic Process Mining Analysis
print("\n" + "="*60)
print("STEP 6: BASIC PROCESS MINING ANALYSIS")
print("="*60)

def basic_process_analysis(df, key_columns):
    """Perform basic process mining analysis"""
    
    results = {}
    
    # 1. Case Analysis
    if key_columns['case_id']:
        case_col = key_columns['case_id']
        print(f"1. CASE ANALYSIS (using '{case_col}')")
        
        n_cases = df[case_col].nunique()
        print(f"Total unique cases: {n_cases:,}")
        
        # Case length distribution
        case_lengths = df.groupby(case_col).size()
        
        print(f"\nCase Length Statistics:")
        print(f"Minimum events per case: {case_lengths.min()}")
        print(f"Maximum events per case: {case_lengths.max()}")
        print(f"Average events per case: {case_lengths.mean():.2f}")
        print(f"Median events per case: {case_lengths.median():.2f}")
        print(f"Standard deviation: {case_lengths.std():.2f}")
        
        # Distribution percentiles
        print(f"\nCase Length Distribution:")
        for p in [10, 25, 50, 75, 90, 95, 99]:
            percentile = case_lengths.quantile(p/100)
            print(f"{p}th percentile: {percentile:.1f} events")
        
        results['case_stats'] = {
            'total_cases': n_cases,
            'case_lengths': case_lengths
        }
    
    # 2. Activity Analysis
    if key_columns['activity']:
        activity_col = key_columns['activity']
        print(f"\n2. ACTIVITY ANALYSIS (using '{activity_col}')")
        
        n_activities = df[activity_col].nunique()
        print(f"   Total unique activities: {n_activities}")
        
        # Frequency of activities
        activity_counts = df[activity_col].value_counts()
        
        print(f"\nTop 15 Most Frequent Activities:")
        top_n = 15
        for i, (activity, count) in enumerate(activity_counts.head(top_n).items(), 1):
            percentage = (count / len(df)) * 100
            print(f"{i:2d}. {str(activity)[:50]:50} {count:9,} ({percentage:.1f}%)")
        
        # Pareto analysis (80/20 rule)
        total_events = len(df)
        cumulative_sum = 0
        pareto_activities = 0
        
        for i, (activity, count) in enumerate(activity_counts.items(), 1):
            cumulative_sum += count
            if cumulative_sum / total_events >= 0.8:
                pareto_activities = i
                break
        
        print(f"\nPareto Analysis:")
        print(f"Top {pareto_activities} activities account for 80% of all events")
        print(f"That's {pareto_activities/n_activities*100:.1f}% of activities causing 80% of work")
        
        results['activity_stats'] = {
            'total_activities': n_activities,
            'activity_counts': activity_counts
        }
    
    # 3. Temporal Analysis
    if '_timestamp' in df.columns and df['_timestamp'].notnull().any():
        print(f"\n3. TEMPORAL ANALYSIS")
        
        # Ensure datetime is properly ordered
        df = df.sort_values('_timestamp')
        
        time_range = df['_timestamp'].max() - df['_timestamp'].min()
        print(f"Time period covered: {time_range.days} days")
        print(f"From: {df['_timestamp'].min()}")
        print(f"To:   {df['_timestamp'].max()}")
        
        # Events over time
        df['_date'] = df['_timestamp'].dt.date
        daily_counts = df.groupby('_date').size()
        
        print(f"\nDaily Event Statistics:")
        print(f"Average events per day: {daily_counts.mean():.0f}")
        print(f"Maximum events in a day: {daily_counts.max():,}")
        print(f"Minimum events in a day: {daily_counts.min():,}")
        print(f"Busiest day: {daily_counts.idxmax()} with {daily_counts.max():,} events")
        
        # Day of week analysis
        df['_dayofweek'] = df['_timestamp'].dt.dayofweek
        df['_dayname'] = df['_timestamp'].dt.day_name()
        
        print(f"\nEvents by Day of Week:")
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        for day in day_order:
            count = (df['_dayname'] == day).sum()
            percentage = (count / len(df)) * 100
            print(f"{day:10}: {count:9,} ({percentage:.1f}%)")
        
        # Hourly analysis
        df['_hour'] = df['_timestamp'].dt.hour
        print(f"\nEvents by Hour of Day:")
        for hour in range(24):
            count = (df['_hour'] == hour).sum()
            if count > 0:
                percentage = (count / len(df)) * 100
                print(f"     {hour:02d}:00 - {hour:02d}:59: {count:9,} ({percentage:.1f}%)")
        
        results['temporal_stats'] = {
            'time_range': time_range,
            'daily_counts': daily_counts
        }
    
    # 4. Resource Analysis (if available)
    if key_columns['resource']:
        resource_col = key_columns['resource']
        print(f"\n4. RESOURCE ANALYSIS (using '{resource_col}')")
        
        n_resources = df[resource_col].nunique()
        print(f"Total unique resources: {n_resources}")
        
        resource_counts = df[resource_col].value_counts()
        
        print(f"\nTop 15 Busiest Resources:")
        for i, (resource, count) in enumerate(resource_counts.head(15).items(), 1):
            percentage = (count / len(df)) * 100
            print(f"{i:2d}. {str(resource)[:40]:40} {count:6,} ({percentage:.1f}%)")
        
        # Resource utilization imbalance
        if len(resource_counts) > 1:
            busiest = resource_counts.iloc[0]
            quietest = resource_counts.iloc[-1]
            imbalance_ratio = busiest / quietest if quietest > 0 else float('inf')
            
            print(f"\nResource Utilization Imbalance:")
            print(f"Busiest resource: {resource_counts.index[0]} with {busiest:,} events")
            print(f"Quietest resource: {resource_counts.index[-1]} with {quietest:,} events")
            print(f"Imbalance ratio: {imbalance_ratio:.1f}x")
        
        results['resource_stats'] = {
            'total_resources': n_resources,
            'resource_counts': resource_counts
        }
    
    return results

# Perform basic analysis
analysis_results = basic_process_analysis(bpi_df, key_columns)



STEP 6: BASIC PROCESS MINING ANALYSIS
1. CASE ANALYSIS (using 'ID')
Total unique cases: 1,595,923

Case Length Statistics:
Minimum events per case: 1
Maximum events per case: 1
Average events per case: 1.00
Median events per case: 1.00
Standard deviation: 0.00

Case Length Distribution:
10th percentile: 1.0 events
25th percentile: 1.0 events
50th percentile: 1.0 events
75th percentile: 1.0 events
90th percentile: 1.0 events
95th percentile: 1.0 events
99th percentile: 1.0 events

2. ACTIVITY ANALYSIS (using 'activity')
   Total unique activities: 42

Top 15 Most Frequent Activities:
 1. Record Goods Receipt                                 314,097 (19.7%)
 2. Create Purchase Order Item                           251,734 (15.8%)
 3. Record Invoice Receipt                               228,760 (14.3%)
 4. Vendor creates invoice                               219,919 (13.8%)
 5. Clear Invoice                                        194,393 (12.2%)
 6. Record Service Entry Sheet              

In [25]:
# Step 7: Advanced Process Mining - Process Variants
print("\n" + "="*60)
print("STEP 7: ADVANCED ANALYSIS - PROCESS VARIANTS")
print("="*60)

def analyze_process_variants(df, key_columns):
    """Analyze different process paths/variants"""
    
    if not (key_columns['case_id'] and key_columns['activity']):
        print("Missing case or activity column for variant analysis")
        return None
    
    case_col = key_columns['case_id']
    activity_col = key_columns['activity']
    
    print(f"Identifying process variants...")
    
    try:
        # Sort by case and timestamp
        if '_timestamp' in df.columns:
            df_sorted = df.sort_values([case_col, '_timestamp'])
        else:
            df_sorted = df.sort_values(case_col)
        
        # Create trace for each case
        print("Creating case traces...")
        traces = df_sorted.groupby(case_col)[activity_col].apply(list)
        
        # Convert trace to string representation
        print("Analyzing trace variants...")
        trace_strings = traces.apply(lambda x: ' -> '.join(map(str, x)))
        
        # Count variants
        variant_counts = trace_strings.value_counts()
        n_variants = len(variant_counts)
        
        print(f"\nProcess Variant Analysis:")
        print(f"Total unique process variants: {n_variants:,}")
        print(f"Average cases per variant: {len(traces) / n_variants:.1f}")
        
        # Most common variants
        print(f"\nTop 10 Most Common Variants:")
        total_cases = len(traces)
        for i, (variant, count) in enumerate(variant_counts.head(10).items(), 1):
            percentage = (count / total_cases) * 100
            
            # Truncate long variants for display
            if len(variant) > 100:
                display_variant = variant[:97] + "..."
            else:
                display_variant = variant
            
            print(f"\n{i:2d}. Variant #{i}")
            print(f"Cases: {count:,} ({percentage:.1f}%)")
            print(f"Path: {display_variant}")
        
        # Variant length analysis
        trace_lengths = traces.apply(len)
        variant_length_stats = trace_lengths.groupby(trace_strings).first()
        
        print(f"\nVariant Length Statistics:")
        print(f"Shortest variant: {variant_length_stats.min()} activities")
        print(f"Longest variant: {variant_length_stats.max()} activities")
        print(f"Average variant length: {variant_length_stats.mean():.1f} activities")
        
        # Complexity analysis
        print(f"\nProcess Complexity Metrics:")
        print(f"Variants covering 50% of cases: {len(variant_counts[variant_counts.cumsum() <= total_cases * 0.5]):,}")
        print(f"Variants covering 80% of cases: {len(variant_counts[variant_counts.cumsum() <= total_cases * 0.8]):,}")
        print(f"Variants covering 95% of cases: {len(variant_counts[variant_counts.cumsum() <= total_cases * 0.95]):,}")
        
        return {
            'traces': traces,
            'variant_counts': variant_counts,
            'variant_length_stats': variant_length_stats
        }
        
    except Exception as e:
        print(f"Error in variant analysis: {str(e)}")
        return None

# Analyze process variants
variant_results = analyze_process_variants(bpi_df, key_columns)



STEP 7: ADVANCED ANALYSIS - PROCESS VARIANTS
Identifying process variants...
Creating case traces...
Analyzing trace variants...

Process Variant Analysis:
Total unique process variants: 42
Average cases per variant: 37998.2

Top 10 Most Common Variants:

 1. Variant #1
Cases: 314,097 (19.7%)
Path: Record Goods Receipt

 2. Variant #2
Cases: 251,734 (15.8%)
Path: Create Purchase Order Item

 3. Variant #3
Cases: 228,760 (14.3%)
Path: Record Invoice Receipt

 4. Variant #4
Cases: 219,919 (13.8%)
Path: Vendor creates invoice

 5. Variant #5
Cases: 194,393 (12.2%)
Path: Clear Invoice

 6. Variant #6
Cases: 164,975 (10.3%)
Path: Record Service Entry Sheet

 7. Variant #7
Cases: 57,136 (3.6%)
Path: Remove Payment Block

 8. Variant #8
Cases: 46,592 (2.9%)
Path: Create Purchase Requisition Item

 9. Variant #9
Cases: 32,065 (2.0%)
Path: Receive Order Confirmation

10. Variant #10
Cases: 21,449 (1.3%)
Path: Change Quantity

Variant Length Statistics:
Shortest variant: 1 activities
Longest va

In [26]:
# Step 8: Performance Analysis
print("\n" + "="*60)
print("STEP 8: PERFORMANCE & DURATION ANALYSIS")
print("="*60)

def analyze_performance(df, key_columns):
    """Analyze performance metrics and bottlenecks"""
    
    if not (key_columns['case_id'] and '_timestamp' in df.columns):
        print("Missing case or timestamp data for performance analysis")
        return None
    
    case_col = key_columns['case_id']
    
    print(f"Calculating case performance metrics...")
    
    try:
        # Ensure proper ordering for timestamps
        df_sorted = df.sort_values([case_col, '_timestamp'])
        
        # Calculate case start and end times
        case_times = df_sorted.groupby(case_col)['_timestamp'].agg(['min', 'max', 'count'])
        case_times.columns = ['start_time', 'end_time', 'event_count']
        
        # Calculate duration
        case_times['duration_hours'] = (case_times['end_time'] - case_times['start_time']).dt.total_seconds() / 3600
        
        print(f"\nCase Duration Analysis:")
        print(f"Cases analyzed: {len(case_times):,}")
        
        # Basic statistics
        duration_stats = case_times['duration_hours'].describe()
        print(f"\nDuration Statistics (hours):")
        print(f"Minimum:    {duration_stats['min']:.1f}")
        print(f"Maximum:    {duration_stats['max']:.1f}")
        print(f"Mean:       {duration_stats['mean']:.1f}")
        print(f"Median:     {case_times['duration_hours'].median():.1f}")
        print(f"Std Dev:    {duration_stats['std']:.1f}")
        
        # Percentile analysis
        print(f"\nDuration Percentiles (hours):")
        for p in [10, 25, 50, 75, 90, 95, 99]:
            percentile = case_times['duration_hours'].quantile(p/100)
            print(f"{p}th percentile: {percentile:.1f}")
        
        # Identify long-running cases
        threshold_95 = case_times['duration_hours'].quantile(0.95)
        long_cases = case_times[case_times['duration_hours'] > threshold_95]
        
        print(f"\nLong-running Cases (top 5%):")
        print(f"Threshold: > {threshold_95:.1f} hours")
        print(f"Number of cases: {len(long_cases):,} ({len(long_cases)/len(case_times)*100:.1f}%)")
        
        if len(long_cases) > 0:
            print(f"Example long case IDs: {long_cases.index[:5].tolist()}")
        
        # Throughput analysis
        print(f"\nProcess Throughput Analysis:")
        
        if len(case_times) > 1 and (case_times['end_time'].max() - case_times['start_time'].min()).days > 0:
            total_days = (case_times['end_time'].max() - case_times['start_time'].min()).days
            avg_cases_per_day = len(case_times) / total_days
            
            print(f"Total days observed: {total_days}")
            print(f"Average cases per day: {avg_cases_per_day:.1f}")
            print(f"Average cycle time: {case_times['duration_hours'].mean() / 24:.1f} days")
        
        # Bottleneck identification (waiting times between activities)
        if key_columns['activity']:
            activity_col = key_columns['activity']
            print(f"\nIdentifying Bottleneck Activities...")
            
            # Calculate waiting time between consecutive events in same case
            df_sorted['next_time'] = df_sorted.groupby(case_col)['_timestamp'].shift(-1)
            df_sorted['wait_time_hours'] = (df_sorted['next_time'] - df_sorted['_timestamp']).dt.total_seconds() / 3600
            
            # Filter out negative wait times (shouldn't happen if sorted correctly)
            valid_waits = df_sorted[df_sorted['wait_time_hours'] >= 0]
            
            # Average wait time per activity
            activity_wait_times = valid_waits.groupby(activity_col)['wait_time_hours'].agg(['mean', 'median', 'std', 'count'])
            activity_wait_times = activity_wait_times.sort_values('mean', ascending=False)
            
            print(f"\nTop 10 Activities with Longest Average Wait Times:")
            for i, (activity, row) in enumerate(activity_wait_times.head(10).iterrows(), 1):
                print(f"{i:2d}. {str(activity)[:50]:50} {row['mean']:6.1f}h (median: {row['median']:.1f}h, n={row['count']:,})")
        
        return {
            'case_times': case_times,
            'duration_stats': duration_stats,
            'activity_wait_times': activity_wait_times if 'activity_wait_times' in locals() else None
        }
        
    except Exception as e:
        print(f"Error in performance analysis: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# Analyze performance
performance_results = analyze_performance(bpi_df, key_columns)



STEP 8: PERFORMANCE & DURATION ANALYSIS
Calculating case performance metrics...

Case Duration Analysis:
Cases analyzed: 1,595,923

Duration Statistics (hours):
Minimum:    0.0
Maximum:    0.0
Mean:       0.0
Median:     0.0
Std Dev:    0.0

Duration Percentiles (hours):
10th percentile: 0.0
25th percentile: 0.0
50th percentile: 0.0
75th percentile: 0.0
90th percentile: 0.0
95th percentile: 0.0
99th percentile: 0.0

Long-running Cases (top 5%):
Threshold: > 0.0 hours
Number of cases: 0 (0.0%)

Process Throughput Analysis:
Total days observed: 26372
Average cases per day: 60.5
Average cycle time: 0.0 days

Identifying Bottleneck Activities...

Top 10 Activities with Longest Average Wait Times:
 1. Block Purchase Order Item                             nanh (median: nanh, n=0.0)
 2. Cancel Goods Receipt                                  nanh (median: nanh, n=0.0)
 3. Cancel Invoice Receipt                                nanh (median: nanh, n=0.0)
 4. Cancel Subsequent Invoice             

In [27]:
# Step 9: Data Quality Assessment
print("\n" + "="*60)
print("STEP 9: DATA QUALITY ASSESSMENT")
print("="*60)

def assess_data_quality(df, key_columns):
    """Assess data quality and identify issues"""
    
    print("Assessing data quality...")
    
    quality_issues = []
    
    # 1. Missing values
    print(f"\n1. Missing Values Analysis:")
    missing_total = df.isnull().sum().sum()
    missing_percentage = (missing_total / (df.shape[0] * df.shape[1])) * 100
    
    print(f"Total missing values: {missing_total:,}")
    print(f"Percentage of all data: {missing_percentage:.2f}%")
    
    if missing_total > 0:
        print(f"\n   Columns with missing values:")
        missing_by_col = df.isnull().sum()
        missing_by_col = missing_by_col[missing_by_col > 0].sort_values(ascending=False)
        
        for col, count in missing_by_col.head(10).items():
            percentage = (count / len(df)) * 100
            print(f"     {col:30} {count:9,} ({percentage:.1f}%)")
    
    # 2. Duplicates
    print(f"\n2. Duplicate Analysis:")
    
    # Exact duplicate rows
    exact_duplicates = df.duplicated().sum()
    print(f"Exact duplicate rows: {exact_duplicates:,} ({exact_duplicates/len(df)*100:.2f}%)")
    
    # Duplicate events within cases
    if key_columns['case_id'] and key_columns['activity'] and '_timestamp' in df.columns:
        case_col = key_columns['case_id']
        activity_col = key_columns['activity']
        
        duplicate_events = df.duplicated(subset=[case_col, activity_col, '_timestamp']).sum()
        print(f"Duplicate events (same case, activity, timestamp): {duplicate_events:,}")
        
        if duplicate_events > 0:
            quality_issues.append(f"Found {duplicate_events:,} duplicate events")
    
    # 3. Timestamp consistency
    print(f"\n3. Timestamp Consistency:")
    
    if '_timestamp' in df.columns:
        # Check for chronological order within cases
        if key_columns['case_id']:
            df_sorted = df.sort_values([key_columns['case_id'], '_timestamp'])
            df_sorted['next_time'] = df_sorted.groupby(key_columns['case_id'])['_timestamp'].shift(-1)
            
            time_violations = (df_sorted['next_time'] < df_sorted['_timestamp']).sum()
            print(f"Timestamp order violations: {time_violations:,}")
            
            if time_violations > 0:
                quality_issues.append(f"Found {time_violations:,} timestamp order violations")
        
        # Check for future dates
        now = pd.Timestamp.now()
        future_dates = df[df['_timestamp'] > now].shape[0]
        print(f"   Future dates: {future_dates:,}")
        
        # Check for unrealistic dates (too far in past)
        min_date = df['_timestamp'].min()
        if min_date.year < 2000:
            print(f"   Very old dates found (earliest: {min_date})")
    
    # 4. Case completeness
    print(f"\n4. Case Completeness:")
    
    if key_columns['case_id']:
        case_col = key_columns['case_id']
        cases_with_single_event = (df.groupby(case_col).size() == 1).sum()
        print(f"Cases with only 1 event: {cases_with_single_event:,}")
        
        if cases_with_single_event > len(df[case_col].unique()) * 0.1:
            quality_issues.append(f"Many cases ({cases_with_single_event:,}) have only 1 event")
    
    # 5. Summary
    print(f"\n" + "="*40)
    print("DATA QUALITY SUMMARY")
    print("="*40)
    
    if quality_issues:
        print("Issues found:")
        for issue in quality_issues:
            print(f"   • {issue}")
    else:
        print("No major data quality issues found")
    
    # Calculate quality score
    quality_score = 100
    
    # Penalize for missing values
    quality_score -= min(20, missing_percentage * 2)
    
    # Penalize for duplicates
    if exact_duplicates > 0:
        duplicate_percentage = (exact_duplicates / len(df)) * 100
        quality_score -= min(15, duplicate_percentage * 3)
    
    # Penalize for time violations
    if time_violations > 0:
        violation_percentage = (time_violations / len(df)) * 100
        quality_score -= min(10, violation_percentage * 5)
    
    quality_score = max(0, quality_score)
    
    print(f"\nOverall Data Quality Score: {quality_score:.1f}/100")
    
    if quality_score >= 90:
        print("Excellent data quality!")
    elif quality_score >= 80:
        print("Good data quality")
    elif quality_score >= 70:
        print("Acceptable data quality, some issues noted")
    else:
        print("Data quality needs improvement")

# Assess data quality
assess_data_quality(bpi_df, key_columns)



STEP 9: DATA QUALITY ASSESSMENT
Assessing data quality...

1. Missing Values Analysis:
Total missing values: 48,882
Percentage of all data: 0.13%

   Columns with missing values:
     cSpendAreaText                    16,294 (1.0%)
     cSpendClassText                   16,294 (1.0%)
     cSubSPendAreaText                 16,294 (1.0%)

2. Duplicate Analysis:
Exact duplicate rows: 0 (0.00%)
Duplicate events (same case, activity, timestamp): 0

3. Timestamp Consistency:
Timestamp order violations: 0
   Future dates: 0
   Very old dates found (earliest: 1948-01-26 22:59:00)

4. Case Completeness:
Cases with only 1 event: 1,595,923

DATA QUALITY SUMMARY
Issues found:
   • Many cases (1,595,923) have only 1 event

Overall Data Quality Score: 99.7/100
Excellent data quality!


In [29]:
# Step 10: Summary and Recommendations
print("\n" + "="*60)
print("STEP 10: SUMMARY & RECOMMENDATIONS")
print("="*60)

def generate_summary_and_recommendations(analysis_results, variant_results, performance_results):
    """Generate summary and actionable recommendations"""
    
    print("ANALYSIS SUMMARY")
    print("-"*40)
    
    if 'case_stats' in analysis_results:
        print(f"Total Cases: {analysis_results['case_stats']['total_cases']:,}")
    
    if 'activity_stats' in analysis_results:
        print(f"Total Activities: {analysis_results['activity_stats']['total_activities']:,}")
    
    if 'temporal_stats' in analysis_results:
        print(f"Time Period: {analysis_results['temporal_stats']['time_range'].days} days")
    
    if performance_results and 'duration_stats' in performance_results:
        print(f"Avg Case Duration: {performance_results['duration_stats']['mean']:.1f} hours")
        print(f"Median Duration: {performance_results['duration_stats']['50%']:.1f} hours")
    
    print("\nACTIONABLE RECOMMENDATIONS")
    print("-"*40)
    
    recommendations = []
    
    # Based on variant analysis
    if variant_results:
        n_variants = len(variant_results['variant_counts'])
        if n_variants > 100:
            recommendations.append("High process variability detected. Consider standardizing process flows.")
    
    # Based on performance analysis
    if performance_results:
        duration_std = performance_results['duration_stats']['std']
        duration_mean = performance_results['duration_stats']['mean']
        
        if duration_std > duration_mean * 0.5:  # High variability
            recommendations.append("High variability in case durations. Investigate root causes.")
        
        # Check for bottlenecks
        if performance_results.get('activity_wait_times') is not None:
            top_bottleneck = performance_results['activity_wait_times'].iloc[0]
            if top_bottleneck['mean'] > 24:  # More than 1 day wait
                activity_name = performance_results['activity_wait_times'].index[0]
                recommendations.append(f"Major bottleneck at '{activity_name}' with {top_bottleneck['mean']:.1f}h average wait. Prioritize optimization.")
    
    # Based on data quality
    if 'resource_stats' in analysis_results:
        resource_counts = analysis_results['resource_stats']['resource_counts']
        if len(resource_counts) > 0:
            busiest = resource_counts.iloc[0]
            quietest = resource_counts.iloc[-1]
            
            if busiest > quietest * 10 and quietest > 0:
                recommendations.append("Significant resource utilization imbalance detected. Consider workload balancing.")
    
    # Based on temporal patterns
    if '_dayofweek' in bpi_df.columns:
        weekend_mask = bpi_df['_dayofweek'] >= 5
        weekend_events = weekend_mask.sum()
        
        if weekend_events > len(bpi_df) * 0.05:  # More than 5% on weekends
            recommendations.append("Significant weekend activity detected. Consider if 24/7 operations are needed or if processes can be optimized.")
    
    # Print recommendations
    if recommendations:
        for i, rec in enumerate(recommendations, 1):
            print(f"{i}. {rec}")
    else:
        print("Process appears to be running efficiently. Maintain current operations.")
    
    print("\nSUGGESTED NEXT STEPS")
    print("-"*40)
    print("1. Implement real-time monitoring for cases exceeding duration thresholds")
    print("2. Create automated alerts for bottleneck activities")
    print("3. Develop dashboard showing: Case throughput, Resource utilization, Bottleneck status")
    print("4. Schedule regular process reviews based on these metrics")
    print("5. Consider process automation for high-frequency, repetitive activities")
    
    print("\nEXPORTING RESULTS")
    print("-"*40)
    
    try:
        # Export key findings to CSV
        export_dir = base_path / "BPI_Analysis_Results"
        export_dir.mkdir(exist_ok=True)
        
        # Export activity frequencies
        if 'activity_stats' in analysis_results:
            activity_df = analysis_results['activity_stats']['activity_counts'].reset_index()
            activity_df.columns = ['Activity', 'Count']
            activity_df['Percentage'] = (activity_df['Count'] / len(bpi_df)) * 100
            activity_df.to_csv(export_dir / "activity_frequencies.csv", index=False)
            print(f"Exported activity frequencies")
        
        # Export case duration statistics
        if performance_results and 'case_times' in performance_results:
            performance_results['case_times'].to_csv(export_dir / "case_durations.csv")
            print(f"Exported case durations")
        
        # Export variant analysis
        if variant_results:
            variants_df = variant_results['variant_counts'].reset_index()
            variants_df.columns = ['Variant_Path', 'Case_Count']
            variants_df['Percentage'] = (variants_df['Case_Count'] / len(variant_results['traces'])) * 100
            variants_df.to_csv(export_dir / "process_variants.csv", index=False)
            print(f"Exported process variants")
        
        print(f"\nAll results exported to: {export_dir}")
        
    except Exception as e:
        print(f"Error exporting results: {str(e)}")

# Generate summary
generate_summary_and_recommendations(analysis_results, variant_results, performance_results)

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)
print("Successfully completed comprehensive BPI 2019 analysis")
print("Analyzed data with proper handling of categorical ordering")
print("Generated actionable insights and recommendations")
print("Results available for further analysis in Tableau")


STEP 10: SUMMARY & RECOMMENDATIONS
ANALYSIS SUMMARY
----------------------------------------
Total Cases: 1,595,923
Total Activities: 42
Time Period: 26372 days
Avg Case Duration: 0.0 hours
Median Duration: 0.0 hours

ACTIONABLE RECOMMENDATIONS
----------------------------------------
1. Significant resource utilization imbalance detected. Consider workload balancing.

SUGGESTED NEXT STEPS
----------------------------------------
1. Implement real-time monitoring for cases exceeding duration thresholds
2. Create automated alerts for bottleneck activities
3. Develop dashboard showing: Case throughput, Resource utilization, Bottleneck status
4. Schedule regular process reviews based on these metrics
5. Consider process automation for high-frequency, repetitive activities

EXPORTING RESULTS
----------------------------------------
Exported activity frequencies
Exported case durations
Exported process variants

All results exported to: C:\Users\user\Portfolio_Exports\BPI_Analysis_Results
